# Project 2 - Earthquake Dataset Download 

*Authors: João Victor Barboza, Thais Lins, Vanessa Uchida, Yuri Martins*

# 1 - Description 

> This notebook consists of requests made to the [USGS - Earthquake Catalog](https://earthquake.usgs.gov/earthquakes/search/) website, so that data from 1980 though to 2018 could be collected and used for future analysis. For this, a few librearies were imported and requests were made on an offset of 20000 (that is the limit value in which the website allows downloads) until all the necessary data was collected. And finally, the datasets were merged together, and will be used later on.

In [0]:
# Import packages
import requests
import pandas as pd
import csv
import io

In [0]:
#Base url used for downloading data
total_results = 209_456
base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=1980-01-01%2000:00:00&endtime=2018-10-26%2023:59:59&minmagnitude=4.5&orderby=time&limit=20000&offset={}'

In [0]:
# Download data
downloaded_count = 0
data = []
for i in range(1, total_results, 20_000):
    data.append(requests.get(base_url.format(i)))

In [0]:
#https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=1980-01-01%2000:00:00&endtime=2018-10-26%2023:59:59&minmagnitude=4.5&orderby=time&limit=20000&offset=0
data

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [0]:
data[0].text[:100]

'time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError'

In [0]:
dfs = [pd.read_csv(io.StringIO(d.content.decode('utf-8'))) for d in data]

In [0]:
#Merge collected dataframes
from functools import reduce

merged_df = reduce(lambda d1, d2: d1.append(d2, ignore_index=True), dfs)
merged_df.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,magError,magNst
count,209457.000000,209457.000000,209457.000000,209457.000000,87625.000000,110710.000000,37466.000000,200202.000000,30235.000000,95990.000000,35817.000000,164656.000000
mean,3.878726,45.582278,72.683005,4.879487,84.961050,95.211482,4.481166,0.978400,8.411724,8.566884,0.102116,27.747231
std,28.925058,120.435723,118.293309,0.411393,99.297185,47.488716,5.941718,0.302074,5.517162,8.375248,0.061556,46.814892
min,-84.133000,-179.999000,-2.540000,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-17.356000,-70.726000,10.450000,4.600000,24.000000,59.000000,1.286000,0.820000,6.100000,3.600000,0.061000,5.000000
50%,-0.066000,103.386000,33.000000,4.800000,48.000000,90.400000,2.687000,0.980000,7.900000,6.400000,0.088000,13.000000
75%,28.109000,142.712000,64.200000,5.000000,103.000000,124.200000,5.047000,1.120000,10.100000,10.700000,0.128000,30.000000
max,87.221000,180.000000,700.900000,9.100000,934.000000,353.000000,62.626000,69.320000,99.000000,791.300000,1.642000,941.000000


In [0]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209457 entries, 0 to 209456
Data columns (total 22 columns):
time               209457 non-null object
latitude           209457 non-null float64
longitude          209457 non-null float64
depth              209457 non-null float64
mag                209457 non-null float64
magType            209456 non-null object
nst                87625 non-null float64
gap                110710 non-null float64
dmin               37466 non-null float64
rms                200202 non-null float64
net                209457 non-null object
id                 209457 non-null object
updated            209457 non-null object
place              209452 non-null object
type               209457 non-null object
horizontalError    30235 non-null float64
depthError         95990 non-null float64
magError           35817 non-null float64
magNst             164656 non-null float64
status             209457 non-null object
locationSource     209457 non-null object


In [0]:
# Export merged dataset to csv
merged_df.to_csv('earthquakes.csv')